In [27]:
import json
from unskript import nbparams
from unskript.fwk.workflow import Task, Workflow
from unskript.secrets import ENV_MODE, ENV_MODE_LOCAL

env = {"ENV_MODE": "ENV_MODE_LOCAL"}
secret_store_cfg = {"SECRET_STORE_TYPE": "SECRET_STORE_TYPE_LOCAL"}

paramDict = {"inputRegion": "us-west-2"}
paramsJson = json.dumps(paramDict)
nbParamsObj = nbparams.NBParams(paramsJson)
inputRegion = nbParamsObj.get('inputRegion')
w = Workflow(env, secret_store_cfg, None, global_vars=globals())

In [28]:
##
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from pydantic import BaseModel, Field
from typing import List
from unskript.connectors.aws import aws_get_paginator
import pprint
from beartype import beartype

from beartype import beartype
@beartype
def aws_get_secrets_manager_secretARN_printer(output):
    if output is None:
        return
    pprint.pprint({"secret": output})


@beartype
@beartype
def aws_get_secrets_manager_secretARN(handle, region: str, secret_name:str) -> str:


    # Create a Secrets Manager client

    client = handle.client(
        service_name='secretsmanager',
        region_name=region
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        # For a list of exceptions thrown, see
        # https://docs.aws.amazon.com/secretsmanager/latest/apireference/API_GetSecretValue.html
        raise e
    #print(get_secret_value_response)
    # Decrypts secret using the associated KMS key.
    secretArn = get_secret_value_response['ARN']
    return secretArn


task = Task(Workflow())
task.configure(credentialsJson='''{
    "credential_name": "aws",
    "credential_type": "CONNECTOR_TYPE_AWS",
    "credential_id": "7aea1127-d810-4d26-bc00-0002dcd23927"
}''')
task.configure(inputParamsJson='''{
    "region": "inputRegion",
    "secret_name": "\\"awsuser-doug-redshift\\""
    }''')
task.configure(outputName="secretArn")

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_get_secrets_manager_secretARN, lego_printer=aws_get_secrets_manager_secretARN_printer, hdl=hdl, args=args)

{'secret': 'arn:aws:secretsmanager:us-west-2:100498623390:secret:awsuser-doug-redshift-KvL4U1'}


In [29]:
import datetime

today = datetime.datetime.now()

yearmonth = today.strftime('%Y%m')
tableName = 'awsbilling'+ yearmonth
todayDay = int(today.strftime('%d'))
yesterDay = 0
if todayDay >1:
    yesterDay = todayDay - 1

sqlQuery = f"SELECT date_part(day, cast(lineitem_usagestartdate as date)) as day, product_instancetype,SUM(lineitem_usageamount)::numeric(37, 4) AS usage_hours, SUM((lineitem_unblendedcost)::numeric(37,4)) AS usage_cost FROM {tableName} WHERE length(lineitem_usagestartdate)>8 AND product_productfamily = 'Compute Instance' AND pricing_unit IN ('Hours', 'Hrs') GROUP BY  day, product_instancetype ORDER BY 1 DESC, 3 DESC, 2 "

print(sqlQuery)

'''
"SELECT date_part(day, cast(lineitem_usagestartdate as date)) as day, product_instancetype,SUM(lineitem_usageamount)::numeric(37, 4) AS usage_hours, SUM((lineitem_unblendedcost)::numeric(37,4)) AS usage_cost FROM awsbilling202302 WHERE length(lineitem_usagestartdate)>8 AND product_productfamily = 'Compute Instance' AND pricing_unit IN ('Hours', 'Hrs') GROUP BY  day, product_instancetype ORDER BY 1 DESC, 3 DESC, 2 "
'''



'\n"SELECT date_part(day, cast(lineitem_usagestartdate as date)) as day, product_instancetype,SUM(lineitem_usageamount)::numeric(37, 4) AS usage_hours, SUM((lineitem_unblendedcost)::numeric(37,4)) AS usage_cost FROM awsbilling202302 WHERE length(lineitem_usagestartdate)>8 AND product_productfamily = \'Compute Instance\' AND pricing_unit IN (\'Hours\', \'Hrs\') GROUP BY  day, product_instancetype ORDER BY 1 DESC, 3 DESC, 2 "\n'

In [36]:
##
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from pydantic import BaseModel, Field
from typing import List, Dict
from unskript.connectors.aws import aws_get_paginator
import pprint


from beartype import beartype

from beartype import beartype

@beartype
def aws_create_redshift_query(handle, region: str,cluster:str, database:str, secretArn: str, query:str) -> str:

    # Input param validation.
    #major change
    client = handle.client('redshift-data', region_name=region)
    # define your query
    query = query
    #query = "SELECT * FROM PG_TABLE_DEF;"
    # execute the query
    response = client.execute_statement(
        ClusterIdentifier=cluster,
        Database=database,
        SecretArn=secretArn,
        Sql=query
    )
    resultId = response['Id']
    #print(response)
    print("resultId",resultId)


    return resultId

#make a change
task = Task(Workflow())
task.configure(credentialsJson='''{}''')
task.configure(inputParamsJson='''{
    "cluster":,
    "database": ,
    "query": "",
    "region": "",
    "secretArn": ""
    }''')
task.configure(outputName="resultId")

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_create_redshift_query,  hdl=hdl, args=args)

resultId 0bf33fee-8682-468d-af85-e7bca14ae03a


In [37]:
##
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from pydantic import BaseModel, Field
from typing import List, Dict
from unskript.connectors.aws import aws_get_paginator
import pprint
from beartype import beartype

from beartype import beartype
@beartype
def aws_get_redshift_query_results_printer(output):
    if output is None:
        return
    pprint.pprint({"QueryDetails": output})


@beartype
@beartype
def aws_get_redshift_query_results(handle, region: str, queryId:str) -> Dict:

    client = handle.client('redshift-data', region_name=region)
    #result = aws_get_paginator(client, 
    #                           "get_statement_result", 
     #                           "TotalNumRows", 
     #                           Id='d7bda35c-7aa4-4414-9272-9e268320df40')
    response = client.describe_statement(
    Id=queryId
    )
    resultReady = response['HasResultSet']
    queryTimeNs = response['Duration']
    ResultRows = response['ResultRows']
    details = {"resultReady": resultReady, 
               "queryTimeNs":queryTimeNs,
               "ResultRows":ResultRows
              }
    
    return details


task = Task(Workflow())
task.configure(credentialsJson='''{}''')
task.configure(inputParamsJson='''{
    "region": "",
    "queryId": ""
    }''')


task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_get_redshift_query_results, lego_printer=aws_get_redshift_query_results_printer, hdl=hdl, args=args)

In [38]:
##
##  Copyright (c) 2023 unSkript, Inc
##  All rights reserved.
##
from pydantic import BaseModel, Field
from typing import List, Dict
from unskript.connectors.aws import aws_get_paginator
import pprint


@beartype
def aws_get_redshift_result(handle, region:str, resultId: str) -> List:


    client = handle.client('redshift-data', region_name=region)
    result = client.get_statement_result(
        Id=resultId
    )
    #result has the Dictionary, but it is not easily queried
    #get all the columns into an array
    columnNames = []
    for column in result['ColumnMetadata']:
        columnNames.append(column['label'])
    #print(columnNames)

    #now let's make the output into a dict
    listResult = []
    for record in result['Records']:

        for key, value in record[0].items():
            rowId = value
        entryCounter = 0
        entryDict = {}
        for entry in record:

            for key, value in entry.items():
                entryDict[columnNames[entryCounter]] = value
            entryCounter +=1
        #print("entryDict",entryDict)
        listResult.append(entryDict)
    return listResult



task = Task(Workflow())
task.configure(credentialsJson='''{}''')
task.configure(inputParamsJson='''{
    "resultId": "",
    "region": ""
    }''')
task.configure(outputName="redshiftresult")

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_get_redshift_result,  hdl=hdl, args=args)

In [39]:
import pandas as pd
import pprint


df = pd.DataFrame.from_dict(redshiftresult)
df['day']=df['day'].astype(int)
df['usage_hours']=df['usage_hours'].astype(float)
df['usage_cost']=df['usage_cost'].astype(float)
#print(df)

In [40]:
import matplotlib as mpl
mpl.use('agg')
from matplotlib.figure import Figure
import panel
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt


font = {'size'   : 16}
plt.rc('font', **font)
dfpivot = df.pivot(index='day', columns='product_instancetype', values='usage_cost')
dfpivot.plot(linewidth=5, ylabel="daily cost in $", figsize=(16, 9), )
plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
plt.show()

dfpivot = df.pivot(index='day', columns='product_instancetype', values='usage_cost')
dfpivot.plot(linewidth=5, ylabel="daily cost in $", figsize=(16, 9), )
plt.ylim((0,10))
plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
plt.show()


In [42]:
today =8
yesterday = today-1
bigchange = {}
for instance in dfpivot.columns:
    todayCost = dfpivot.at[today, instance]
    yesterdayCost = dfpivot.at[yesterday, instance]
    delta =(todayCost-yesterdayCost)/yesterdayCost
    
    if delta >.05:
        #print( instance, delta,dfpivot.at[today, instance], dfpivot.at[yesterday, instance])
        bigchange[instance] = {"delta":delta, "todayCost":todayCost,"yesterdayCost":yesterdayCost}
    #elif delta <-.15:
        #print("savings!", instance, delta,dfpivot.at[today, instance], dfpivot.at[yesterday, instance])
        #bigchange[instance] = {"delta":delta, "todayCost":todayCost,"yesterdayCost":yesterdayCost}

    
print("bigchange", bigchange)

bigchange {}
